In [11]:
import numpy as np
import os
seq_dim = (10,10,5)

In [12]:
# model_dir = "logs-gaussian/CNNhiddendim64"
model_dir = "logs-dir/testloss3_CNN64"
# work_dir = os.path.join(model_dir, f"val_time0_scale_T0overTk/epoch{10098}")
work_dir = os.path.join(model_dir, f"val_baseline/epoch{10098}")
seq = np.load(os.path.join(work_dir, "seq_val_step0.npy"))
print(seq.shape)

(400, 10, 10, 5)


In [13]:
logits = np.load(os.path.join(work_dir, "logits_val_step0.npy"))
print(logits.shape)

(400, 2, 10, 10, 5)


In [14]:
import os
from ase import Atoms
from ase.io import write
from copy import deepcopy

In [15]:
elems = [1,2]
nvars = [500]
L = nvars[0]
channel = len(elems)
B = seq.shape[0]

In [16]:

buffer_dir = "data/Al-Cu"
cell = np.load(os.path.join(buffer_dir, "buffer_cell.npy")).reshape(-1,3,3)[0]
_coord = np.load(os.path.join(buffer_dir, "buffer_coord.npy"))
coord = np.transpose(_coord.reshape(-1,3, nvars[0]), (0,2,1))[0]
logits_buffer = np.load(os.path.join(buffer_dir, "buffer_atypes.npy")).reshape(-1,channel,nvars[0])
seq_buffer = np.argmax(logits_buffer, axis=1)
print(logits_buffer.shape, seq_buffer.shape)

(2475, 2, 500) (2475, 500)


In [17]:
def remove_element(atoms, element):
    """ 
    Remove atoms of a specific element from the Atoms object.
    """
    # atomic_numbers = atoms.get_atomic_numbers()
    # new_atomic_numbers = [13 if i == 2 else 29 for i in atomic_numbers]
    # atoms.set_atomic_numbers(new_atomic_numbers)
    indices_to_remove = [i for i, n in enumerate(atoms.get_atomic_numbers()) if n == element]
    del atoms[indices_to_remove]

In [18]:
atype_map = {0: "Al", 1: "Cu"}
seq = seq.reshape(-1,L)
B = len(seq)
anames = [[atype_map[seq[j][i]] for i in range(L)] for j in range(B)]
xyz_file = os.path.join(work_dir, "val-Cu.xyz")
if os.path.exists(xyz_file): os.remove(xyz_file)
for i in range(B):
    atoms = deepcopy(Atoms(anames[i], positions=coord, cell=cell, pbc=True))
    remove_element(atoms, 13)
    write(xyz_file, atoms, append=True)
